In [30]:
# -*- coding: utf-8 -*-
"""
Created on 2020.2.12 11:00
@author: Johnson

"""
from ckiptagger import data_utils, construct_dictionary, WS,POS, NER
import csv
import logging
import gc

In [31]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 載入 ckiptagger 自訂詞典
User_Dict = {}
with open("KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 10    # 未給定權重者一律賦予預設值 10                
dictionary = construct_dictionary(User_Dict)

ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [11]:
csvIn = open("KCC Data/News-CIS Data/NewsCis18850.csv", 'r', newline='', encoding='utf-8-sig')
#for row in csvIn:
    #print(row)

rowlists = csv.reader(csvIn)
rowlists

In [3]:
csvIn = open("KCC Data/News-CIS Data/NewsCis18850.csv", 'r', newline='', encoding='utf-8-sig')
output1 = open("KCC Data/News-CIS Data/NewsCis18850Test.txt", 'w', encoding='utf-8-sig')

# 載入同義字(ex黃捷=黃議員捷等等)
combine_dict = {}
for line in open("KCC Data/Dict/KccSynonym2020.txt", "r", encoding="utf-8-sig"):  
    seperate_word = line.strip().split("\t")
    num = len(seperate_word)
    for i in range(1, num):
        combine_dict[seperate_word[i]] = seperate_word[0] #任何多數個同義字都會等於第一個(以第一個名稱為代表)

# 載入 StopWord
stopword_lst = []
with open("KCC Data/Dict/KccStopWord2020.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_lst.append(stopword.strip())

print("------------CKIPTagger Word Segment ws(str,....) + 合併同義字 + 刪除標點及停用字 ---------------")
rowlists = csv.reader(csvIn)
i = 0
for row in rowlists:
    # 刪除標點, 使用自訂強制詞典
    strCut = ws([row[2]], 
                   sentence_segmentation=True,
                   segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"},
                   coerce_dictionary = dictionary)
    for word in strCut[0]:
        # 合併同義字
        if word in combine_dict:
            word = combine_dict[word]
        
        # 刪除停用字
        if word not in stopword_lst:
            output1.write(word + ' ')
    output1.write('\n')                
    i += 1
    if i % 10 == 0:
        logging.info("已完成CkipCis  %d 檔案 的斷詞" % (i + 1))
output1.close()
csvIn.close()



------------CKIPTagger Word Segment ws(str,....) + 合併同義字 + 刪除標點及停用字 ---------------


2020-06-01 14:49:46,752 : INFO : 已完成CkipCis  11 檔案 的斷詞


KeyboardInterrupt: 

In [32]:
import pandas
df = pandas.read_csv("KCC Data/News-CIS Data/NewsCis18850Test.txt",header = None)
df

,0
0,馬英九 昨天 上午 視察 衛武營藝術文化中心 文化部 長 洪孟啟 工程 副主委 顏久榮 高雄...
1,日本 九州熊本 縣 接連 遭 強震 襲擊 台灣 捐贈 熊本 縣政府 日本 政府 款項 高雄市...
2,壽山動物園 增進 動物飼養 福祉 保育 工作 特 設立 高雄市壽山動物園動物認養辦法 設立 ...
3,日本 熊本地震 災情 慘重 高雄市 長 陳菊 昨天 說 率先 捐出 一月 所得 指示 市府 ...
4,睽違 超過 世紀 緬甸 官方 代表 行 包含 位 緬甸 聯邦 議會 議員 美國 農村 發展 ...
5,高雄 翻轉 須 新 政府 力挺 高雄市 長 陳菊 日前 高雄市議會 做 施政報告 時 指出 ...
6,高雄市 長 陳菊 昨 前往 東台 精機 參訪 高雄 鼓勵 傳統 產業 升級 研發 提升 產業...
7,交通局 委託 民間 公司 民調 昨天 公布 民調 月 搭乘 公車 民眾 滿意度 高達 九十三...
8,臺灣塑膠工業股份有限公司 企業 回饋 地方 藝文 活動 週六 晚間 高雄市 仁武運動公園 登...
9,中國國民黨 分區 立委 陳宜民 鳳山 地區 成立 聯合 服務處 日前 揭 牌 成立 服務處 ...


In [33]:
print("------------CKIPTagger pos(str,....) ---------------------")
pos_sentence_list = pos(strCut)
print(pos_sentence_list)

print("------------CKIPTagger ner(str,....) ---------------------")
entity_sentence_list = ner(strCut, pos_sentence_list)

print(entity_sentence_list)

del ws
del WS
del pos
del POS
del ner
del NER
gc.collect()


------------CKIPTagger pos(str,....) ---------------------
[['D', 'VE', 'VH', 'VH', 'Na', 'D', 'VC', 'Nb', 'Na', 'VH', 'COMMACATEGORY', 'Nc', 'Caa', 'Nc', 'Caa', 'Nc', 'D', 'VE', 'Nb', 'VC', 'Nb', 'PERIODCATEGORY', 'Na', 'SHI', 'Na', 'COMMACATEGORY', 'P', 'Nh', 'VA', 'VC', 'Na', 'D', 'D', 'VG', 'VH', 'COMMACATEGORY', 'Cbb', 'Nh', 'Nh', 'VJ', 'Na', 'COMMACATEGORY', 'D', 'VJ', 'Na', 'DE', 'Na', 'COMMACATEGORY', 'VE', 'Na', 'VH', 'VK', 'SHI', 'Na', 'VA', 'COMMACATEGORY', 'D', 'VH', 'T', 'PERIODCATEGORY', 'Na', 'VC', 'Na', 'SHI', 'Neu', 'Nf', 'Na', 'PAUSECATEGORY', 'D', 'SHI', 'Neu', 'Nf', 'VJ', 'Na', 'PERIODCATEGORY', 'P', 'Na', 'Na', 'VC', 'COMMACATEGORY', 'VC', 'D', 'P', 'VE', 'SHI', 'VJ', 'VJ', 'Na', 'COMMACATEGORY', 'Cbb', 'VH', 'Ng', 'Na', 'DE', 'VC', 'D', 'SHI', 'VJ', 'Na', 'VE', 'COMMACATEGORY', 'VA', 'Na', 'P', 'VJ', 'D', 'VC', 'Na', 'PAUSECATEGORY', 'VHC', 'Na', 'VH', 'COMMACATEGORY', 'VC', 'VH', 'VH', 'VH', 'DE', 'VK', 'PERIODCATEGORY', 'P', 'COMMACATEGORY', 'Nd', 'Ng', 'COMMACA

860212